In [1]:
from initialization.wp_get_page import WikipediaPage
from initialization.wp_structured_text import WikipediaStructuredText
from initialization.wp_structured_diffs import Diff
from lakat.submit import content_submit
from config.scrape_cfg import WIKIPEDIA_API_URL, EXAMPLE_ARTICLE_TITLE
from config.bucket_cfg import DEFAULT_ATOMIC_BUCKET_SCHEMA, DEFAULT_MOLECULAR_BUCKET_SCHEMA, DEFAULT_NAME_RESOLUTION_BUCKET_SCHEMA, BUCKET_ID_TYPE_NO_REF, BUCKET_ID_TYPE_WITH_ID_REF
from config.db_cfg import TRIE_TYPE
from utils.trie.trie import MerkleTrie
from db.mockdb import MOCK_DB
from config.db_cfg import DB_FOLDER
from config.env import DEV_CROP_FILENAME_AFTER
import os
from setup import db_trie as db_tr
from utils.signing.sign import get_public_key_from_file
from utils.serialize import unserialize, serialize
from initialization.test_branch_creation import branch_creation
from initialization.test_wp_submissions import (first_submit, submit_new_edit)


In [2]:
# db_tr.db.close()

In [3]:

# db.close()
article_title = EXAMPLE_ARTICLE_TITLE
print('article_title: ', article_title)

wp = WikipediaPage(WIKIPEDIA_API_URL)
edit_history = wp.load_content_from_batches(
    article_title, 0, 105, download_if_not_exist=True)
edit_history.reverse()

key_file_prefix="lakat"
public_key = get_public_key_from_file(key_file_prefix=key_file_prefix)
# create a name registry NR (optional)

article_title:  Compartmental models in epidemiology


In [4]:
branchId = branch_creation(public_key=public_key, verbose=False)

In [5]:
new_part_id_to_bucket_id, res = first_submit(edit=edit_history[0], public_key=public_key, branchId=branchId)


In [6]:
article_id = res["bucket_ids"][-1] ## hack TODO! Change this.

In [7]:
new_part_id_to_bucket_id, res = submit_new_edit(
    public_key, 
    branchId, prev_edit=edit_history[0], new_edit=edit_history[1], article_id=article_id, part_id_to_bucket_id=new_part_id_to_bucket_id)

In [8]:

new_part_id_to_bucket_id, res = submit_new_edit(
    public_key, 
    branchId, prev_edit=edit_history[1], new_edit=edit_history[2], article_id=article_id, part_id_to_bucket_id=new_part_id_to_bucket_id)

### Create a new mock db from the old one

In [14]:


db_name = "test_2"

# db = DB(name=DB_NAME)
db_path = os.path.abspath(os.path.join(os.getcwd(), DB_FOLDER))
db_mock = MOCK_DB(path=db_path, name=db_name, create=True, crop_filename_after=DEV_CROP_FILENAME_AFTER)
# TODO: SHOULD be created only on branch creation
another_trie = MerkleTrie(db=db_tr.db, branchId="")

In [15]:
another_trie.load_trie_from_db(root_hash='QmeqQn')

In [22]:
another_trie.get_json_representation()

{'516d5': {'a': {'6': {'3': {'3761': '{"Qmdt93": "Compartmental models in epidemiology"}'},
    'f': {'5969': b''}},
   '4': {'e5468': b''}},
  '8': {'47544a': b''},
  '2': {'5': {'2': {'3478': 'QmY6yg'}, '6': {'3351': b''}},
   '6': {'b6463': b''}},
  '4': {'6': {'2': {'3374': b''}, '8': {'7563': b''}}, '4': {'13254': b''}},
  '3': {'636343': b''},
  '9': {'37': {'5869': b''}, '36': {'7967': b''}},
  '0': {'4c516b': b''}},
 '516d6': {'6': {'39': {'6762': b''}, '35': {'6854': b''}},
  '1': {'76367a': b''},
  '5': {'4': {'45a6e': b''}, '6': {'15543': b''}},
  '4': {'743933': b''},
  '2': {'6b': {'3736': b''}, '65': {'6971': b''}}},
 '516d4': {'e6d4479': b''}}

In [9]:
res

{'bucket_ids': ['QmTtoe', 'QmdhqV'],
 'molecule_ids': ['QmdhqV'],
 'branch_id': 'QmRR4x',
 'branch_state': 'QmZCW4',
 'submit_id': 'QmTjMx',
 'submit_trac_id': 'QmfLdp',
 'registered_names': [],
 'nr_regs': -1,
 'name_registration_deployed': False,
 'msg': "/* Carrier state */ Image tweak. By '82.148.33.8'."}

In [11]:
db_tr.db.get(key='QmeqQn'.encode('utf-8'), entry_type=TRIE_TYPE)

b'{"children": {"5": "QmTFQa"}, "value": null, "interaction": null, "path": ""}'